In [ ]:
%matplotlib inline

# Mahalanobis distance

Here is an example of the FCM with a different distance metric. I recommend you read on the GK as well.

In [ ]:
from IPython import display
import pylab as pl
from matplotlib.patches import Ellipse
from sklearn import datasets
import numpy as np
import time
from sklearn import datasets

# import some data to play with
whichone = 0
if( whichone == 0 ):
    X, y = datasets.make_blobs(n_samples=1000,
                             centers=[[0.0, 0.0],[1.0, 1.0],[3.0, 2.0],[5.0, 5.0],[3.0,5.0]],
                             cluster_std=[0.1, 0.1, 0.1, 0.1, 0.2],
                             random_state=8)
elif( whichone == 2 ):
    X, y = datasets.make_classification(n_samples=500, n_features=2, n_informative=2, 
                    n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=1,
                    class_sep=2.5, flip_y=0, weights=[0.5,0.5])   
print( "Shape of data:", X.shape )
N = X.shape[0]
D = X.shape[1]

# lets pick prototype's from the data randomly
C = 5
Prototypes = np.zeros((C,D))
for i in range(C):
    WhichSample = np.random.randint(0, N)
    Prototypes[i,:] = X[WhichSample,:]
print( "Prototypes start" )
print( Prototypes )
# lets give them identity covariances
Covs = np.zeros((C,D,D))
for i in range(C):
    Covs[i,:,:] = np.eye(D) * (np.random.random() + 0.1)
print( "First cov" )
print( Covs[0,:,:] )

# this is our membership matrix
MembMatrix = np.zeros((N,C))

# fuzzy factor
M = 1.5

# run that algorithm
T = 100
fig, ax = pl.subplots(nrows = 1, ncols = 1, figsize=(7, 5))
for t in range(T):
                
    pl.clf()        
        
    # compute distances (doing it long winded to show you)
    X_P_DistMatrix = np.zeros((C,X.shape[0]))
    for i in range(C):
        for j in range(X.shape[0]):
            diff = X[j,:] - Prototypes[i,:]
            A = Covs[i,:,:] + (np.eye(D) * 0.00001)
            B = np.linalg.inv(A) * (np.linalg.det(A)**(1/D)) * (1.0**(1/D))
            calc2 = np.matmul(diff.transpose(),B)
            calc3 = np.matmul(calc2,diff)
            X_P_DistMatrix[i,j] = np.sqrt( calc3 )
            
    # update memb matrix
    for i in range(C):
        for n in range(N):
            sumv = 0
            top = X_P_DistMatrix[i,n]
            for m in range(C):                
                bottom = X_P_DistMatrix[m,n]
                sumv = sumv + pow(top / (bottom + np.finfo(float).eps),1.0/(M-1.0))
            MembMatrix[n,i] = min(max(1.0 / (sumv + np.finfo(float).eps),0.0),1.0)            
    
    # update protos
    for c in range(C):
        
        # mean
        Top = np.zeros((1,D))
        Bottom = 0        
        for n in range(N):
            Top = Top + ( X[n,:] * pow(MembMatrix[n,c],M) )
            Bottom = Bottom + pow(MembMatrix[n,c],M)
        Prototypes[c,:] = Top / (Bottom + np.finfo(float).eps)
        
        # cov
        Top = np.zeros((D,D))
        Bottom = 0        
        for n in range(N):
            diff = X[n,:]-Prototypes[c,:]
            Top = Top + ( np.outer(diff,diff) * pow(MembMatrix[n,c],M) )
            Bottom = Bottom + pow(MembMatrix[n,c],M)
        Covs[c,:,:] = Top / (Bottom + np.finfo(float).eps)        
        
        # drawing time
        
        v, w = np.linalg.eigh( Covs[c,:,:] )
        u = w[0] / np.linalg.norm(w[0])
        anglee = np.arctan2(u[1], u[0])
        anglee = 180 * anglee / np.pi  # convert to degrees
        v = 1. * np.sqrt(2.) * np.sqrt(v)
        ell = Ellipse(Prototypes[c,:], v[0], v[1], 180 + anglee, facecolor='r', edgecolor='k', alpha=0.5 )
        pl.gca().add_artist(ell)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        ell = Ellipse(Prototypes[c,:], v[0], v[1], 180 + anglee, facecolor='g', edgecolor='k', alpha=0.3 )
        pl.gca().add_artist(ell)        
        v = 3. * np.sqrt(2.) * np.sqrt(v)
        ell = Ellipse(Prototypes[c,:], v[0], v[1], 180 + anglee, facecolor='c', edgecolor='k', alpha=0.15 )
        pl.gca().add_artist(ell) 
        
    # plot    
    X1 = X[:,0]
    X2 = X[:,1]
    pl.scatter(X1, X2, edgecolor='b', alpha=0.3 )  
    pl.plot(Prototypes[:,0],Prototypes[:,1],'xr')
        
    # animation, so pause it!
    display.clear_output(wait=True)
    display.display(pl.gcf())
    time.sleep(0.01)    
    
print( "done" )